In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

namespace OrderManagementSystem
{
    public enum OrderStatus
    {
        Created,
        InProgress,
        Completed
    }

    public class Company
    {
        public string Name { get; set; }
        public List<Employee> Employees { get; private set; } = new List<Employee>();
        public List<Order> Orders { get; private set; } = new List<Order>();

        public event Action<Order> OrderStatusChanged;

        public void AddEmployee(Employee employee)
        {
            Employees.Add(employee);
        }

        public void AddOrder(Order order)
        {
            order.OrderId = Orders.Count + 1;
            Orders.Add(order);
        }

        public void AssignEmployeeToOrder(int orderId, Employee employee)
        {
            var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
            if (order != null)
            {
                order.AssignedEmployee = employee;
                Console.WriteLine($"Order {orderId} assigned to {employee.FirstName} {employee.LastName}");
            }
        }

        public void ChangeOrderStatus(int orderId, OrderStatus status)
        {
            var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
            if (order != null)
            {
                order.Status = status;
                OrderStatusChanged?.Invoke(order);

                if (order.AssignedEmployee != null && status == OrderStatus.Completed)
                {
                    order.AssignedEmployee.OrdersProcessed++;
                }
            }
        }

        public IEnumerable<Order> GetOrdersByStatus(OrderStatus status)
        {
            return Orders.Where(o => o.Status == status);
        }

        public IEnumerable<Order> GetOrdersByCustomer(string customerFirstName, string customerLastName)
        {
            return Orders.Where(o => o.Customer.FirstName == customerFirstName && o.Customer.LastName == customerLastName);
        }

        public void GenerateEmployeeReport()
        {
            foreach (var employee in Employees)
            {
                Console.WriteLine($"{employee.FirstName} {employee.LastName} - Processed Orders: {employee.OrdersProcessed}");
            }
        }
    }

    public class Employee
    {
        public string FirstName { get; set; }
        public string LastName { get; set; }
        public string Position { get; set; }
        public int OrdersProcessed { get; set; }
    }

    public class Customer
    {
        public string FirstName { get; set; }
        public string LastName { get; set; }
        public string PhoneNumber { get; set; }
    }

    public class Order
    {
        public int OrderId { get; set; }
        public string Description { get; set; }
        public Employee AssignedEmployee { get; set; }
        public OrderStatus Status { get; set; } = OrderStatus.Created;
        public DateTime CreatedAt { get; private set; } = DateTime.Now;
        public Customer Customer { get; set; }
    }

    class Program
    {
        static void Main(string[] args)
        {
            var company = new Company { Name = "TechCorp" };

            // Подписка на событие изменения статуса заказа
            company.OrderStatusChanged += order =>
            {
                Console.WriteLine($"Order {order.OrderId} status changed to {order.Status}");
            };

            // Создаем сотрудников
            var employee1 = new Employee { FirstName = "John", LastName = "Doe", Position = "Manager" };
            var employee2 = new Employee { FirstName = "Jane", LastName = "Smith", Position = "Developer" };

            company.AddEmployee(employee1);
            company.AddEmployee(employee2);

            // Создаем заказчиков
            var customer1 = new Customer { FirstName = "Alice", LastName = "Brown", PhoneNumber = "123-456-7890" };
            var customer2 = new Customer { FirstName = "Bob", LastName = "Johnson", PhoneNumber = "098-765-4321" };

            // Создаем заказы
            var order1 = new Order { Description = "Website Development", Customer = customer1 };
            var order2 = new Order { Description = "Mobile App", Customer = customer2 };

            company.AddOrder(order1);
            company.AddOrder(order2);

            // Назначаем сотрудника на заказ
            company.AssignEmployeeToOrder(1, employee1);

            // Изменяем статус заказа
            company.ChangeOrderStatus(1, OrderStatus.InProgress);
            company.ChangeOrderStatus(1, OrderStatus.Completed);

            // Просмотр заказов по статусу
            var completedOrders = company.GetOrdersByStatus(OrderStatus.Completed);
            Console.WriteLine("Completed Orders:");
            foreach (var order in completedOrders)
            {
                Console.WriteLine($"Order {order.OrderId}: {order.Description}");
            }

            // Генерация отчета по сотрудникам
            company.GenerateEmployeeReport();
        }
    }
}

Error: (5,1): error CS7021: Cannot declare namespace in script code